In [14]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

from sklearn.model_selection import train_test_split

In [26]:
df = pd.read_csv("arxiv-dataset-processed-L1.csv")

In [12]:
bert_preprocess = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
bert_encoder = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [28]:
#Split into train (80%) and test set (20%)
X = df.iloc[:,1]
y = df.iloc[:,2]
y.head()

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(61766,) (15442,) (61766,) (15442,)


In [32]:
text_input = tf.keras.Input(shape=(), dtype = tf.string, name="text")
preprocessing_layer = hub.KerasLayer(bert_preprocess, name = "preprocessing")
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer(bert_encoder, name = "bert_encoder")
outputs = encoder(encoder_inputs)
net = outputs["pooled_output"]
net = tf.keras.layers.Dropout(0.1)(net)
net = tf.keras.layers.Dense(8, activation = "sigmoid", name = "classifier")(net)
model = tf.keras.Model(text_input, net)

In [34]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                

In [35]:
#Train and test
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "rmsprop", metrics = ["acc"])

In [39]:
history = model.fit(X_train, y_train, epochs = 5)

Epoch 1/5
  61/1931 [..............................] - ETA: 3:44:56 - loss: 1.5139 - acc: 0.4395

KeyboardInterrupt: 

0